In [1]:
import pandas as pd

import os
import glob

In [2]:
def merge_dir(dir_name: str):
    dir_path = f'{os.getcwd()}/data/{dir_name}/'
    
    people_df = pd.read_csv(dir_path + 'PEOPLE.csv', index_col='temp_id')
    people_df = people_df.rename(columns={'url': 'person_url'})
    
    address_people =  pd.read_csv(dir_path + 'ADDRESSES_PEOPLE.csv')
    
    df = address_people.merge(people_df, how='left', left_on='person_uuid', right_on='temp_id', right_index=True).set_index('person_uuid', drop=True)
    
    df['copy_index'] = df.index
    
    addresses_df = pd.read_csv(dir_path + 'ADDRESSES.csv', index_col='temp_id')
    addresses_df = addresses_df.drop(columns=['countries_scope'])
    addresses_df = addresses_df.rename(columns={'url': 'address_url'})
    
    df = df.merge(addresses_df, how='left', left_on='address_uuid', right_on='temp_id').set_index('copy_index')

    df['copy_index'] = df.index
    
    people_publications = pd.read_csv(dir_path + 'PEOPLE_PUBLICATIONS.csv')
    
    df = df.merge(people_publications, how='left', left_index=True, right_on='person_uuid')
    
    publications_df = pd.read_csv(dir_path + 'PUBLICATIONS.csv', index_col='temp_id')
    publications_df = publications_df.rename(columns={'url': 'publication_url'})
    
    df = df.merge(publications_df, left_on='publication_uuid', right_on='temp_id').reset_index()
    
    df = df.drop(['address_uuid', 'publication_uuid', 'person_uuid', 'copy_index'], axis=1)
    
    return df
    

In [3]:
df = merge_dir(0)
df

,index,lastname,firstname,countries_scope,email,town,role,person_url,address,lat,lon,name,phone,address_url,title,pubmed_id,journal,year,publication_url,abstract
0,0,Andersen,Lars B,NO,NaN,sogndal,NaN,NaN,"sogn og fjordane university collegesogndal, no...",NaN,NaN,faculty of teacher education and sport,NaN,NaN,Cardiorespiratory fitness cut points to avoid ...,27670254,British journal of sports medicine. Volume: 50...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27670254/,Poor cardiorespiratory fitness is associated w...
1,1,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"university of bergenbergen, norway",NaN,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
2,2,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,department of pathology,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
3,3,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,department of pathology,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
4,4,Collett,Karin,NO,NaN,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,department of pathology,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,1991,Nord,E,NO,NaN,oslo,NaN,NaN,"national institute of public healthoslo, norway",NaN,NaN,NaN,NaN,NaN,The trade-off between severity of illness and ...,10128127,"Health policy (Amsterdam, Netherlands). Volume...",1993.0,http://www.ncbi.nlm.nih.gov/pubmed/10128127/,Social appreciation of health care programs is...
1992,1992,Jorde,Rolf,NO,rolf.jorde@unn.no,tromsø,NaN,NaN,"university of tromsøtromsø, norway",NaN,NaN,"department of internal medicine, institute of ...",NaN,NaN,Neuropsychological function in relation to ser...,16283099,Journal of neurology. Volume: 253 Issue: 4 Apr...,2006.0,http://www.ncbi.nlm.nih.gov/pubmed/16283099/,There are receptors for parathyroid hormone (P...
1993,1993,Engedal,Knut,NO,NaN,oslo,NaN,NaN,"oslo university hospitaloslo, norway",NaN,NaN,NaN,NaN,NaN,Erratum to: Access to timely formal dementia c...,27793151,BMC health services research. Volume: 16 Issue...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27793151/,NaN
1994,1994,Selbaek,Geir,NO,NaN,oslo,NaN,NaN,"oslo university hospitaloslo, norway",NaN,NaN,NaN,NaN,NaN,Erratum to: Access to timely formal dementia c...,27793151,BMC health services research. Volume: 16 Issue...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27793151/,NaN


In [4]:
df = pd.concat((merge_dir(i) for i in range(50)), axis=0, ignore_index=True)

In [5]:
df = df.drop_duplicates(['firstname', 'lastname', 'pubmed_id'])
df

,index,lastname,firstname,countries_scope,email,town,role,person_url,address,lat,lon,name,phone,address_url,title,pubmed_id,journal,year,publication_url,abstract
0,0,Andersen,Lars B,NO,NaN,sogndal,NaN,NaN,"sogn og fjordane university collegesogndal, no...",NaN,NaN,faculty of teacher education and sport,NaN,NaN,Cardiorespiratory fitness cut points to avoid ...,27670254,British journal of sports medicine. Volume: 50...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27670254/,Poor cardiorespiratory fitness is associated w...
1,1,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"university of bergenbergen, norway",NaN,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
4,4,Collett,Karin,NO,NaN,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,department of pathology,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
6,6,Arnes,Jarle,NO,NaN,bergen,NaN,NaN,"university of bergenbergen, norway",NaN,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
9,9,Akslen,Lars A,NO,NaN,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,department of pathology,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96541,1903,Omfjord,Christina S,NO,csandvand@hotmail.com.,kristiansand,NaN,NaN,"university of agderkristiansand, norway",NaN,NaN,"department of public health, sport and nutrition",NaN,NaN,Physical education Teachers' and public health...,26704344,BMC public health. Volume: 15 Dec 24 2015,2015.0,http://www.ncbi.nlm.nih.gov/pubmed/26704344/,High quality physical education programs in hi...
96542,1904,Rohde,Gudrun,NO,gudrun.e.rohde@uia.no.,kristiansand,NaN,NaN,"university of agderkristiansand, norway",NaN,NaN,"department of public health, sport and nutrition",NaN,NaN,Physical education Teachers' and public health...,26704344,BMC public health. Volume: 15 Dec 24 2015,2015.0,http://www.ncbi.nlm.nih.gov/pubmed/26704344/,High quality physical education programs in hi...
96543,1905,Kiserud,T,NO,torvid.kiserud@kk.uib.no,bergen,NaN,NaN,"haukeland university hospitalbergen, norway",NaN,NaN,"department of clinical medicine, university of...",NaN,NaN,Global reproductive health: is diagnostic ultr...,22911621,Ultrasound in obstetrics & gynecology : the of...,2012.0,http://www.ncbi.nlm.nih.gov/pubmed/22911621/,NaN
96544,1906,Anderssen,Norman,NO,norman.anderssen@psych.uib.no,university of bergen,NaN,NaN,department of psychosocial scienceuniversity o...,NaN,NaN,NaN,NaN,NaN,Outcomes for children with lesbian or gay pare...,12361102,Scandinavian journal of psychology. Volume: 43...,2002.0,http://www.ncbi.nlm.nih.gov/pubmed/12361102/,Twenty-three empirical studies published betwe...
